# Introduction
This Jupyter Notebook serves as a demonstration of how Pyomo can be used for optimizing an EV charging network. The goal is to showcase the capabilities of Pyomo in solving complex optimization problems related to EV charging infrastructure.


Find the `py_env_setup` folder in the repository, then:
- For advanced Python programmer: 

    **requirements.txt** is provided to set up the EVCSAP project environment. Copy it in your system user folder (e.g., C:\Users\z004ffpm). The link below provides a way of using this for environment set up
 > https://stackoverflow.com/questions/48787250/set-up-virtualenv-using-a-requirements-txt-generated-by-conda
 
 
 
- For others: (anaconda is required for set up)
    1. Read the **Spec List** or **Environment.yml** section of this blog https://www.anaconda.com/blog/moving-conda-environments. 
    2. Copy either `EVCSAP_env_list.yml`, or `OptPyomoSP.txt` (depending on which file you use) in your system user folder where the anaconda can find the environment file. (e.g., C:\Users\z004ffpm)


Furthermore: 
> - If `geopy` is missing, execute `pip install geopy` in (anaconda) prompt
> - If `MPI-SPPy` is missing, use `conda install openmpi`. Then,`conda install mpi4py` and finally `pip install mpi-sppy` in (anaconda) prompt

In [ ]:
# Detect current folder to avoid package import error
import os, sys
currentdir = os.path.dirname(os.path.realpath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(currentdir)

## Step 1: Pre-process Geo Data (Grid Connections Excluded)

Pre-process to get the Set-up Dictionary for building up **MPDP** frame model which calculates expressions like distances between nodes and EESC, etc. 

In [ ]:
# To import Opensource packages
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
import time
import pickle
from datetime import datetime

import idaes
import numpy as np
import pyomo.environ as pyo

from pyomo.core.util import quicksum 
from pyomo.core.expr.current import evaluate_expression 
# Used for calculating distance for nodes with (lat, lon) 
from geopy.distance import geodesic 


In [ ]:
m_name = 'test_model'
model = pyo.ConcreteModel(name = m_name )
print(f"Built an empty concrete pyomo model named {model.name}. \nDefining sets   ...")        

## Setup Sets:

In [ ]:
model.I_update = pyo.Set(
    initialize = [1, 2, 3],
    # initialize = setup_dict['model_sets']['I_update'], 
    # domain = pyo.NonNegativeIntegers,
    doc = 'Candidate locations with EXISITING CSs and CPs.'
)


model.I_newBuild = pyo.Set(
    initialize = [4,5,6],
    # domain = pyo.NonNegativeIntegers,  
    doc = 'Candidate locations for NEW CS.'
) 


model.J = pyo.Set(
    initialize = ['a', 'b', 'c'],
    # domain = pyo.NonNegativeIntegers ,    
    doc = 'Set of CD centers.'
    )


model.I = model.I_update | model.I_newBuild # All candidate Locs

model.T = pyo.Set(
    initialize = ['spring', 'winter'],
    # domain = pyo.NonNegativeIntegers ,    
    doc = 'Set of time periods.'
    )

In [ ]:
model.display()

In [ ]:
model.I.display()

## Setup Param.

In [ ]:
# model.nodes = pyo.Param(
#     (model.I | model.J), 
#     initialize = [(1,2), (3,4), (4,5)],
#     within = pyo.Any,
#     doc = """Coordinates of all nodes including candilocs and CD centers, used for calculating walking distances"""
# )

# model.N_newCPs = pyo.Param(
#     initialize = setup_dict['params_basic']['N_newCPs'],
#     domain = pyo.NonNegativeIntegers,
#     mutable = True, 
#     # , default = 14
#     doc = 'Maximum total amount of new CPs the investors want to install.'
# )


## Setup DVs.

In [ ]:
model.x = pyo.Var(
    model.I, 
    domain = pyo.NonNegativeReals, 
    initialize = 0.50,
    bounds = (0.20, 1),
    doc = '''This is the charging price for each CS i \in I'''
)

In [ ]:
model.x.display()

In [ ]:
model.gross_revenue = quicksum(
        model.calD[i,j,t] * model.tau[i,j] * model.calA[j,t] \
            * model.z[i,j,t] * model.u[i] \
            for i in model.I for j in model.J for t in model.T) * 365

model.cs_placement_cost = quicksum(
    model.c_x[i] * model.x[i] + model.c_y * model.y[i] \
        for i in model.I
)

model.obj_profit_no_grid = pyo.Objective(
        rule = model.gross_revenue - model.cs_placement_cost , 
        sense = pyo.maximize
    )

In [ ]:
def MaxUpdateCSs(model):
    return quicksum(model.x[i] for i in model.I_update) <= model.N_update
model.MaxUpdateCSs = pyo.Constraint(rule = MaxUpdateCSs)

## Explore Pyomo with Copilot


In [ ]:

cost_values = {}
for s in model.S:
    for t in model.T:
        cost_values[s, t] = random.uniform(0, 1.5)


In [ ]:
cost_values

In [ ]:
from pyomo.environ import *
import random

# Create a model
model = ConcreteModel()

Data_dict = {
    'sites': ["DE_1", "DE_3", "DK_1", "DK5"],
    "time_periods": ["q1", "q2", "q3", "q4"],
    'total_cost': {
        ('DE_1', 'q1'): 0.028829095554299522,
        ('DE_1', 'q2'): 0.7671715340421157,
        ('DE_1', 'q3'): 1.4437029176258833,
        ('DE_1', 'q4'): 0.9897958906603088,
        ('DE_3', 'q1'): 1.1486292371906317,
        ('DE_3', 'q2'): 0.43728232345616846,
        ('DE_3', 'q3'): 0.7880991557824376,
        ('DE_3', 'q4'): 0.32991921148343106,
        ('DK_1', 'q1'): 0.9430381789907805,
        ('DK_1', 'q2'): 0.2895551785573069,
        ('DK_1', 'q3'): 0.8605290573576347,
        ('DK_1', 'q4'): 0.812620950130807,
        ('DK5', 'q1'): 1.2666696520561975,
        ('DK5', 'q2'): 0.7934970428028281,
        ('DK5', 'q3'): 1.167442259088418,
        ('DK5', 'q4'): 0.5121052265062618
        },
    'p_min': 0.1,
    'p_max': 0.5,
    'd_max': 100
}


# Define sets
model.S = Set(initialize=Data_dict['sites'])
model.T = Set(initialize=Data_dict['time_periods'])

# Define variables
model.d = Var(
    model.S, 
    model.T, 
    bounds=(0, Data_dict['d_max']),
    domain=NonNegativeReals,
    doc = 'dv: cd'
)

model.p = Var(
    model.S, 
    model.T,  
    bounds=(Data_dict['p_min'], Data_dict['p_max']),
    domain=NonNegativeReals,
    doc = 'dv: price'
)

# Define parameters
model.c = Param(
    model.S, 
    model.T, 
    initialize=Data_dict['total_cost'], 
    doc = 'param: cost'
)
# model.p_min = Param(initialize=p_min)
# model.p_max = Param(initialize=p_max)
# model.d_max = Param(initialize=d_max)

# Define objective
model.profit = Objective(
    expr = sum(
        model.d[s,t]*(model.p[s,t]-model.c[s,t]) \
            for s in model.S for t in model.T
        ), 
    sense = maximize
)

# Define constraints
# def price_constraints_rule(model, s, t):
#     return model.p_min <= model.p[s,t] <= model.p_max
# model.price_constraints = Constraint(model.S, model.T, rule=price_constraints_rule)

# def demand_constraints_rule(model, s):
#     return model.d[s,t] <= model.d_max
# model.demand_constraints = Constraint(model.S, rule=demand_constraints_rule)


In [ ]:
model.profit.display()

In [ ]:
from pyomo.opt import SolverFactory

# Create a solver instance
# solver = SolverFactory('glpk')
solver = SolverFactory('ipopt')
solver = SolverFactory('cplex')

# Solve the model
result = solver.solve(model)

# # Check the solver status
# if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
#     # The model was solved to optimality
#     print("Optimal solution found.")
#     # Access the variable values
#     for var in model.component_data_objects(Var):
#         print(f"{var.name}: {var.value}")
# else:
#     # The model failed to solveca
#     print("Solver did not find an optimal solution.")
